In [1]:
suppressMessages(library(rwwa))
png_res <- 240

nsamp <- 1000

# specify statistical model per variable
mdl_args <- list("rx1day-ondjfm" = list(dist = "gev", type = "fixeddisp", lower = F),
                 "rx90day" = list(dist = "norm_logt", type = "shift", lower = F))

# standardised return periods per variable & region
return_periods <- list("rx1day-ondjfm_s" = 40, "rx1day-ondjfm_n" = 5,
                       "rx90day_s" = 10, "rx90day_n" = 10)

# covariates to loop over when fitting model variants
cov_list <- list("gmst-only" = "gmst", "gmst+nao" = c("gmst", "nao"))

# get observed NAO (already standardised) to use as factual covariate
nao_obs <- load_ts("ts-obs/med-storms_nao-djf_era5-stn.dat", col.names = c("year", "nao"))
nao_2026 <- nao_obs[nao_obs$year == 2026,"nao"]

# CORDEX


## Trend fitting

In [ ]:
nsamp <- 1000

for (mtype in names(cov_list)[2]) {
    covnm <- cov_list[[mtype]]
    for (varnm in c("rx1day-ondjfm", "rx90day")[1]) {
        for (rnm in c("s", "n")[1]) {
            rp <- return_periods[[paste0(varnm,"_",rnm)]]
            fl <- list.files("ts-cordex", pattern = paste0(varnm, "_",rnm), full.names = T)

            for (fnm in fl) {

                print(fnm)
                gcm <- paste0(strsplit(fnm, "_")[[1]][5:6], collapse = "_")
                res_fnm <- gsub(".dat", paste0("_",mtype,".csv"),gsub("ts-cordex/med-storms", "res-cordex/res-cordex", fnm))
                if (file.exists(res_fnm)) next
                
                ts <- load_ts(fnm, col.names = c("year", "x"))
        
                gmst_fnm <- list.files("~/00_WWA_project_folder/live/data/cmip5_covariates/cmip5-gmst", 
                                       pattern = gsub("_", "_rcp85_", gsub("i1p1",".dat",gcm)), full.names = T)
                if (length(gmst_fnm) == 0) {
                    print("     No GMST")
                    next
                }
                new_gmst_fnm <- paste0("ts-cordex/med-storms_gmst_cmip5_",gcm,".dat")
                if (!file.exists(new_gmst_fnm)) file.copy(gmst_fnm, new_gmst_fnm)
                
                nao_fnm <- list.files("ts-cordex", pattern = paste0("nao-djf_cmip5_",gcm,".dat"), full.names = T)
                if (length(nao_fnm) == 0) {
                    print("     No NAO")
                    next
                }
        
                # create dataframe & detrend NAO
                gmst <- load_ts(new_gmst_fnm, col.names = c("year", "gmst"))
                nao <- load_ts(nao_fnm, col.names = c("year", "nao"))
                df <- merge(merge(gmst, nao), ts)
                # df$nao_res <- resid(lm(nao ~ gmst, df))
        
                # use model's GMST & observed NAO state to define factual climate
                gmst_2026 <- df$gmst[df$year == 2026]
                cov_2026 <- data.frame(gmst = gmst_2026, "nao" = nao_2026)
                
                cov_cf <- rbind("pi" = cov_2026 - c(1.3,0),
                                "naoneutral" = c(gmst_2026, 0),
                                "pineutral" = c(gmst_2026-1.3,0))
                cov_fut <- rbind("fut" = cov_2026 + c(1.3,0))

                # fit template model
                mdl <- do.call(fit_ns, append(mdl_args[[varnm]], list("data" = df, "varnm" = "x", covnm = covnm)))

                # bootstrap results
                res <- cmodel_results(mdl, rp = rp, cov_f = cov_2026, cov_hist = cov_cf, cov_fut = cov_fut, y_now = 2026, y_start = 1979, y_fut = 2070, nsamp = nsamp)

                # bootstrap correlation between NAO & precip in this region
                set.seed(42)
                corr_boot <- sapply(1:nsamp, function(i) {
                    cor(df[sample(1:nrow(df), replace = T), c("nao", "x")])[1,2]
                })
                corr_qq <- data.frame("naocorr_est" = cor(df$nao, df$x), "naocorr_lower" = quantile(corr_boot,0.025), "naocorr_upper" = quantile(corr_boot,0.975))
                res <- cbind(res, corr_qq)
                write.csv(res, res_fnm)

                # quick return level plots to check fit
                rlplot_fnm <- gsub("csv", "png", gsub("res-cordex/res-cordex", "mdl-eval/rlplot", res_fnm))
                if(!file.exists(rlplot_fnm)) {
                    nc <- 1 + length(covnm) * 2
                    png(rlplot_fnm, h = png_res, w = nc * png_res*1.25); {
                        prep_window(c(1,nc), oma = c(0,0,2,0))
                        plot_trend(mdl)
                        plot_covtrend(mdl, "gmst")
                        if (nc == 5) { plot_covtrend(mdl, covnm[2]) }
                        plot_returnlevels(mdl, cov_f = cov_2026, cov_cf = cov_cf["pi",,drop = F], nsamp = 50, legend_pos = "bottomright")
                        if (nc == 5) { plot_returnlevels(mdl, cov_f = cov_2026, cov_cf = cov_cf["naoneutral",,drop = F], nsamp = 50, legend_pos = "bottomright") }
                        mtext(gsub(".+cordex_|.csv","",res_fnm), side = 3, outer = T, font = 2, line = -1)
                    }; dev.off()
                }
            }
        }
    }
}

## Compile

In [4]:
varnm <- "rx90day"
rnm <- "s"

mdl_res <- t(sapply(list.files("res-highresmip", pattern = paste0(varnm, "_", rnm, ".+_gmst\\+nao"), full.names = T), read.csv))
rownames(mdl_res) <- sapply(strsplit(gsub(".csv","",rownames(mdl_res)), "_"), "[", 4)
write.csv(mdl_res, paste0("res-highresmip_",varnm,"_",rnm,"_gmst+nao.csv"))

# mdl_res <- t(sapply(list.files("res-highresmip", pattern = paste0(varnm, "_", rnm, ".+_gmst-only"), full.names = T), read.csv))
# rownames(mdl_res) <- sapply(strsplit(gsub(".csv","",rownames(mdl_res)), "_"), "[", 4)
# write.csv(mdl_res, paste0("res-highresmip_",varnm,"_",rnm,"_gmst-only.csv"))

## Run NAO/precip correlations for all models

Original version didn't assign the output to a variable so needs to be done separately


In [74]:
for (varnm in c("rx1day-ondjfm", "rx90day")[2]) {
    for (rnm in c("s", "n")) {
        corr_fnm <- paste0("nao-corrs_",varnm, "_", rnm,".csv")
        fl <- list.files("ts-cordex", pattern = paste0(varnm, "_",rnm), full.names = T)
        corrs <- sapply(fl, function(fnm) {
            gcm <- paste0(strsplit(fnm, "_")[[1]][5:6], collapse = "_")
            # corr_fnm <- gsub(".dat", paste0("_",mtype,".csv"),gsub("ts-cordex/med-storms", "res-cordex/nao-corr", fnm))
            # if (file.exists(corr_fnm)) next
            
            ts <- load_ts(fnm, col.names = c("year", "x"))
            nao_fnm <- list.files("ts-cordex", pattern = paste0("nao-djf_cmip5_",gcm,".dat"), full.names = T)
            if (length(nao_fnm) == 0) { return(data.frame(t(rep(NA, 3)))) }
    
            # create dataframe & detrend NAO
            nao <- load_ts(nao_fnm, col.names = c("year", "nao"))
            df <- merge(load_ts(nao_fnm, col.names = c("year", "nao")), ts)

            # filter to only recent historic period
            df <- df[df$year >= 1950 & df$year <= 2026,]
            df <- df[!is.na(df$x),]

            # bootstrap correlation between NAO & precip in this region
            set.seed(42)
            corr_boot <- sapply(1:1000, function(i) {
                cor(df[sample(1:nrow(df), replace = T), c("nao", "x")])[1,2]
            })
            res <- data.frame(t(c(cor(df$nao, df$x), quantile(corr_boot,c(0.025, 0.975)))))
        }, simplify = F)
        corrs <- data.frame(rbind.fill(corrs), row.names = gsub(".dat", "", apply(sapply(strsplit(names(corrs), "_"), "[", 5:7), 2, paste0, collapse = "_")))
        colnames(corrs) <- c("est", "lower", "upper")
        corrs <- corrs[!is.na(corrs$est),1:3]
        write.csv(corrs, corr_fnm)
    }
}

# HighResMIP


## Trend fitting

In [ ]:
# rx1day, s, GMST + NAO
nsamp <- 1000

for (mtype in names(cov_list)[2]) {
    covnm <- cov_list[[mtype]]
    for (varnm in c("rx1day-ondjfm", "rx90day")) {
        for (rnm in c("s", "n")) {
            rp <- return_periods[[paste0(varnm,"_",rnm)]]
            fl <- list.files("ts-highresmip", pattern = paste0(varnm, "_",rnm), full.names = T)

            for (fnm in fl) {

                print(fnm)
                gcm <- gsub(".+_|.dat","",fnm)
                res_fnm <- gsub(".dat", paste0("_",mtype,".csv"),gsub("ts-highresmip/med-storms", "res-highresmip/res-highresmip", fnm))
                if (file.exists(res_fnm)) next
                
                ts <- load_ts(fnm, col.names = c("year", "x"))
        
                gmst_fnm <- list.files("~/00_WWA_project_folder/live/data/highresmip/GSAT/", pattern = paste0(gcm, "_"), full.names = T)
                if (length(gmst_fnm) > 1) { gmst_fnm <- gmst_fnm[grepl("r1i1p1f1", gmst_fnm)] }
                if (length(gmst_fnm) == 0) {
                    print("     No GMST")
                    next
                }
                
                new_gmst_fnm <- paste0("ts-highresmip/med-storms_gmst_",gcm,".dat")
                if (!file.exists(new_gmst_fnm)) file.copy(gmst_fnm, new_gmst_fnm)
                
                nao_fnm <- list.files("ts-highresmip", pattern = paste0("nao-djf_",gcm,".dat"), full.names = T)
                if (length(nao_fnm) == 0) {
                    print("     No NAO")
                    next
                }
        
                # create dataframe
                gmst <- load_ts(new_gmst_fnm, col.names = c("year", "gmst"))
                nao <- load_ts(nao_fnm, col.names = c("year", "nao"))
                df <- merge(merge(gmst, nao), ts)
        
                # use model's GMST & observed NAO state to define factual climate
                gmst_2026 <- df$gmst[df$year == 2026]
                cov_2026 <- data.frame(gmst = gmst_2026, "nao" = nao_2026)
                
                cov_cf <- rbind("pi" = cov_2026 - c(1.3,0),
                                "naoneutral" = c(gmst_2026, 0),
                                "pineutral" = c(gmst_2026-1.3,0))
                cov_fut <- rbind("fut" = cov_2026 + c(1.3,0))

                # fit template model
                mdl <- do.call(fit_ns, append(mdl_args[[varnm]], list("data" = df, "varnm" = "x", covnm = covnm)))

                # bootstrap results
                res <- cmodel_results(mdl, rp = rp, cov_f = cov_2026, cov_hist = cov_cf, cov_fut = cov_fut, y_now = 2026, y_start = 1979, y_fut = 2070, nsamp = nsamp)

                # bootstrap correlation between NAO & precip in this region
                set.seed(42)
                corr_boot <- sapply(1:nsamp, function(i) {
                    cor(df[sample(1:nrow(df), replace = T), c("nao", "x")])[1,2]
                })
                corr_qq <- data.frame("naocorr_est" = cor(df$nao, df$x), "naocorr_lower" = quantile(corr_boot,0.025), "naocorr_upper" = quantile(corr_boot,0.975))
                res <- cbind(res, corr_qq)
                write.csv(res, res_fnm)

                # quick return level plots to check fit
                rlplot_fnm <- gsub("csv", "png", gsub("res-highresmip/res-highresmip", "mdl-eval/rlplot", res_fnm))
                if(!file.exists(rlplot_fnm)) {
                    nc <- 1 + length(covnm) * 2
                    png(rlplot_fnm, h = png_res, w = nc * png_res*1.25); {
                        prep_window(c(1,nc), oma = c(0,0,2,0))
                        plot_trend(mdl)
                        plot_covtrend(mdl, "gmst")
                        if (nc == 5) { plot_covtrend(mdl, covnm[2]) }
                        plot_returnlevels(mdl, cov_f = cov_2026, cov_cf = cov_cf["pi",,drop = F], nsamp = 50, legend_pos = "bottomright")
                        if (nc == 5) { plot_returnlevels(mdl, cov_f = cov_2026, cov_cf = cov_cf["naoneutral",,drop = F], nsamp = 50, legend_pos = "bottomright") }
                        mtext(gsub(".+highresmip_|.csv","",res_fnm), side = 3, outer = T, font = 2, line = -1)
                    }; dev.off()
                }
            }
        }
    }
}

[1] "ts-highresmip/med-storms_rx1day-ondjfm_s_CMCC-CM2-HR4.dat"
[1] "ts-highresmip/med-storms_rx1day-ondjfm_s_CMCC-CM2-VHR4.dat"
[1] "ts-highresmip/med-storms_rx1day-ondjfm_s_CNRM-CM6-1-HR.dat"
[1] "ts-highresmip/med-storms_rx1day-ondjfm_s_CNRM-CM6-1.dat"
[1] "ts-highresmip/med-storms_rx1day-ondjfm_s_EC-Earth3P-HR.dat"
[1] "ts-highresmip/med-storms_rx1day-ondjfm_s_EC-Earth3P.dat"
[1] "ts-highresmip/med-storms_rx1day-ondjfm_s_HadGEM3-GC31-HM.dat"
[1] "ts-highresmip/med-storms_rx1day-ondjfm_s_HadGEM3-GC31-LM.dat"
[1] "ts-highresmip/med-storms_rx1day-ondjfm_s_HadGEM3-GC31-MM.dat"
[1] "ts-highresmip/med-storms_rx1day-ondjfm_s_MPI-ESM1-2-HR.dat"
[1] "ts-highresmip/med-storms_rx1day-ondjfm_s_MPI-ESM1-2-XR.dat"
[1] "ts-highresmip/med-storms_rx1day-ondjfm_n_CMCC-CM2-HR4.dat"
[1] "ts-highresmip/med-storms_rx1day-ondjfm_n_CMCC-CM2-VHR4.dat"
[1] "ts-highresmip/med-storms_rx1day-ondjfm_n_CNRM-CM6-1-HR.dat"
[1] "ts-highresmip/med-storms_rx1day-ondjfm_n_CNRM-CM6-1.dat"
[1] "ts-highresmip/med-storms_